In [1]:
import numpy as np

In [ ]:
class Node:
    def __init__(self,left=None, right=None, value=None, threshold=None, feature=None):
        self.left = left
        self.right = right
        self.threshold = threshold
        self.value = value
        self.feature = feature
     
    

In [ ]:
class Decision_Tree:
    def __init__(self):

    def fit(self,X,y):

    def predict(self,X):
        